
# ABE Communication & QKD — Interactive Demos

This notebook adds two practical demos:

1. **Aharonov–Bohm (AB) Phase Modulation for Communication**  
   Encode bits by changing enclosed flux \( \Phi \) → phase shift \( \Delta\phi_{\rm AB} \) → fringe shift.  
   Receiver demodulates by correlating against reference fringes. We estimate BER vs. noise.

2. **Quantum Key Distribution (BB84) Toy Model**  
   Simulate basis choices, sifting, QBER, and eavesdropper (intercept–resend) effect.


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from math import pi
rng = np.random.default_rng(123)

def newfig():
    plt.figure(figsize=(7,4))



## 1) AB Phase-Modulated Communication

- Transmitter: maps bit 0 → \( \Phi_0 \) fraction 0.00, bit 1 → fraction 0.25.  
- Interference intensity: \( I(x) = I_0[1 + V \cos(kx + \Delta\phi)] \) with \( \Delta\phi = 2\pi (\Phi/\Phi_0) \).  
- Channel noise: additive Gaussian on sampled intensities.  
- Receiver: correlates samples with two references (bit0, bit1) and decides by max correlation.


In [ ]:

# Parameters
I0, V, k = 1.0, 0.9, 20.0
q = 1.602e-19; h = 6.62607015e-34
phi0 = h/q
x = np.linspace(-2*np.pi, 2*np.pi, 400)  # sample points on screen

dphi0 = 2*np.pi*(0.00)       # bit 0
dphi1 = 2*np.pi*(0.25)       # bit 1

I_ref0 = I0*(1 + V*np.cos(k*x + dphi0))
I_ref1 = I0*(1 + V*np.cos(k*x + dphi1))

def tx_bit(bit):
    dphi = dphi0 if bit==0 else dphi1
    return I0*(1 + V*np.cos(k*x + dphi))

def add_noise(I, sigma=0.1):
    return I + rng.normal(0.0, sigma, size=I.shape)

def decide(Irx):
    # correlate vs references
    c0 = np.dot(Irx, I_ref0)
    c1 = np.dot(Irx, I_ref1)
    return 0 if c0>=c1 else 1

# quick demo
bits = rng.integers(0,2, size=16)
sigma = 0.15
rx = [add_noise(tx_bit(b), sigma) for b in bits]
dec = [decide(r) for r in rx]
ber = np.mean(bits!=np.array(dec))
print("Demo BER @ sigma=%.2f:"%sigma, ber)

newfig()
plt.plot(x, I_ref0, label='Ref bit 0')
plt.plot(x, I_ref1, label='Ref bit 1')
plt.xlabel('x'); plt.ylabel('Intensity'); plt.title('AB References (phase-coded)'); plt.legend(); plt.tight_layout(); plt.show()


In [ ]:

# BER vs Noise sweep
sigmas = np.linspace(0.0, 0.4, 21)
BER = []
Nbits = 200
for s in sigmas:
    b = rng.integers(0,2, size=Nbits)
    rx = [add_noise(tx_bit(bb), s) for bb in b]
    dec = [decide(r) for r in rx]
    BER.append(np.mean(b!=np.array(dec)))
newfig(); plt.plot(sigmas, BER, marker='o')
plt.xlabel('Noise σ'); plt.ylabel('Bit Error Rate'); plt.title('AB Phase Modulation: BER vs Noise'); plt.tight_layout(); plt.show()



## 2) BB84 QKD Toy Model

Steps:
1. Alice chooses random bits and bases \( \mathcal{Z}=\{|0\rangle,|1\rangle\}, \mathcal{X}=\{|+\rangle,|-\rangle\} \).
2. Bob measures in random bases.
3. They announce bases, **sift** matching positions → raw key.
4. Compute **QBER** from a random sample (or full set here).
5. Optional Eve (intercept–resend with prob. \(p\)) introduces errors.


In [ ]:

def bb84(N=2000, p_eve=0.0, seed=123):
    rng = np.random.default_rng(seed)
    # 0 = Z basis, 1 = X basis
    A_bits = rng.integers(0,2,size=N)
    A_bases = rng.integers(0,2,size=N)
    B_bases = rng.integers(0,2,size=N)

    # ideal channel w/out Eve: Bob's result matches Alice if same basis; random if different
    B_results = np.zeros(N, dtype=int)

    for i in range(N):
        a_bit, a_basis = A_bits[i], A_bases[i]
        b_basis = B_bases[i]

        # Eve intercept-resend with probability p_eve
        if rng.random() < p_eve:
            # Eve picks random basis to measure Alice's qubit
            e_basis = rng.integers(0,2)
            if e_basis == a_basis:
                e_bit = a_bit
            else:
                e_bit = rng.integers(0,2)
            # Eve resends state |e_bit, e_basis>
            # Bob measures that; if his basis matches e_basis, he gets e_bit, else random
            if b_basis == e_basis:
                B_results[i] = e_bit
            else:
                B_results[i] = rng.integers(0,2)
        else:
            # No Eve
            if b_basis == a_basis:
                B_results[i] = a_bit
            else:
                B_results[i] = rng.integers(0,2)

    # Sifting
    keep = (A_bases == B_bases)
    A_key = A_bits[keep]
    B_key = B_results[keep]
    if len(A_key)==0:
        return 0.0, 0, 0
    QBER = np.mean(A_key != B_key)
    return QBER, len(A_key), N

# Sweep Eve probability
Ps = np.linspace(0, 1.0, 11)
qbers = []
sizes = []
for p in Ps:
    q, m, N = bb84(N=4000, p_eve=p, seed=42)
    qbers.append(q); sizes.append(m)
newfig(); plt.plot(Ps, qbers, marker='o')
plt.xlabel('Intercept–Resend probability p_eve'); plt.ylabel('QBER')
plt.title('BB84: QBER vs Eavesdropping Probability'); plt.tight_layout(); plt.show()

newfig(); plt.plot(Ps, np.array(sizes)/N, marker='o')
plt.xlabel('p_eve'); plt.ylabel('Sifted fraction (|matching bases| / N)')
plt.title('BB84: Sifted Key Fraction (independent of Eve)'); plt.tight_layout(); plt.show()
